# CNN

In [1]:
import os, random
import numpy as np
import nibabel as nib
from utilities import *

# Pytorch functions
import torch
# Neural network layers
import torch.nn as nn
# Optimizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
# Torchvision library
from torchvision import transforms

In [2]:
# Device configuration
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cpu


In [5]:
def set_seed(seed, use_cuda = True, use_mps = False):
    """
    Set SEED for PyTorch reproducibility
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    if use_mps:
        torch.mps.manual_seed(seed)

SEED = 44

USE_SEED = True

if USE_SEED:
    set_seed(SEED, torch.cuda.is_available(), torch.backends.mps.is_available())

# Transformations

These are some transformations I have written but need to be updated for our dataset.

In [6]:
class Crop(object):
    def __init__(self, output_ind):
        self.output_ind = output_ind
    def __call__(self, sample):
        image, label = sample
        new_image = []
        output_ind = self.output_ind
        for i in range(len(image)): # 4
            new_image.append(image[i][output_ind[0][0]:output_ind[0][1], output_ind[1][0]:output_ind[1][1],:])
            new_label = label[output_ind[0][0]:output_ind[0][1], output_ind[1][0]:output_ind[1][1],:]
        return new_image, new_label

class Flatten(object):
    def __call__(self, sample):
        image, label = sample # images have 4 image
        new_image = []
        for i in range(len(image)):
            new_image.append(image[i].reshape(180, -1, order = 'F'))
        new_label = label.reshape(-1)
        return new_image, new_label
    
class ScanNormalize(object):
    def __call__(self, sample):
        image, label = sample
        new_image = []
        for i in range(len(image)):
            img = image[i]
            new_scan = (img-np.min(img))/(np.max(img)-np.min(img))
            new_image.append(new_scan)
        return new_image, label

class StackScans(object):
    def __call__(self, sample):
        image, label = sample
        new_image = np.stack(image, axis=-1)
        return new_image, label
    
class BinaryLabel(object):
    def __call__(self, sample):
        image, label = sample
        new_label = np.sign(label)
        return image, new_label
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = image.transpose((2, 0, 1))
        return torch.from_numpy(image), torch.from_numpy(landmarks)

# First CNN

We load flair, t1ce, t2 and define any pre-processing methods here. Without preprocessing, the training data has size `(3, 240, 240, 155)` and the target has size `(240, 240, 155)`.

In [49]:
class BraTSDataset(Dataset):
    def __init__(self, image_path = r'./BraTS/BraTS2021_Training_Data', transform=None):
        'Initialisation'
        self.image_path = image_path
        self.folders_name = [folder for folder in os.listdir(self.image_path) if folder != '.DS_Store']
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.folders_name)

    def __getitem__(self, index):
        'Generates one sample of data'

        # Select sample
        fld_name = self.folders_name[index]
        image = []
        for scan_type in ['flair', 't1ce', 't2']:
            path_img = os.path.join(self.image_path, fld_name, fld_name + '_' + scan_type + '.nii.gz')
            img = nib.load(path_img).get_fdata()
            image.append(img)
        
        image = np.array(image)

        path_label = os.path.join(self.image_path, fld_name, fld_name + '_seg.nii.gz')

        label = nib.load(path_label).get_fdata()
        if self.transform:
            image, label = self.transform([image, label])
        return image, label

In [53]:
class everythirdlayer(object):
    def __call__(self, sample):
        image, label = sample
        new_image = image[:,:,:,np.arange(3, 152, 3)]
        new_label = label[:,:,np.arange(3, 152, 3)]
        return new_image, new_label

In [54]:
dataset = BraTSDataset(image_path = r'./BraTS/BraTS2021_Training_Data',
                                    transform=transforms.Compose([
                                        everythirdlayer()
                                    ]))

In [55]:
train0, label0 = dataset[0]
print(train0.shape, label0.shape)

(3, 240, 240, 50) (240, 240, 50)


In [6]:
crop_ind = [[35,215],[10,230]]
crop_len = [crop_ind[0][1]-crop_ind[0][0], crop_ind[1][1]-crop_ind[1][0]]
scan_depth = 155

# Train, val, test split and define DataLoader

In [ ]:
batch_size = 1

train_val_test_split = [0.7, 0.2, 0.1]

generator = torch.Generator().manual_seed(SEED)

dataset_size = len(dataset)
dataset_indices = list(range(dataset_size))

train_sampler, val_sampler, test_sampler = random_split(dataset_indices, train_val_test_split, generator=generator)

In [ ]:
train_loader = DataLoader(dataset, batch_size=batch_size,
                            sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size,
                            sampler=val_sampler)
test_loader = DataLoader(dataset, batch_size=batch_size, sampler = test_sampler)

# CNN Model Structure

3D Images of size 4 155 180 220 -> 4 180 155 * 220 -> 4 `crop_len[0]` `scan_depth`*`crop_len[1]`

Return 155 180 220 -> `scan_depth` `crop_len[0]` `crop_len[1]`

In [ ]:
class FirstCNN(nn.Module):
  def __init__(self, output_dim):
    super().__init__()

    self.features = nn.Sequential(
      nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5),
      nn.MaxPool2d(kernel_size=2),
      nn.ReLU(),
      nn.Conv2d(in_channels=16, out_channels=64, kernel_size=5),
      nn.MaxPool2d(kernel_size=2),
      nn.ReLU(),
      nn.Conv2d(in_channels=64, out_channels=256, kernel_size=5),
      nn.MaxPool2d(kernel_size=2),
      nn.ReLU()
    )
    
    self.linear = nn.Sequential(
      nn.Linear(256 * 5 * 5 * 5, 383625),
      nn.ReLU(),
      nn.Linear(383625, 1534500),
      nn.ReLU(),
      nn.Linear(1534500, output_dim)
    )
    
  def forward(self, x):
    x = self.features(x)
    x = x.view(x.shape[0], -1)
    x = self.linear(x)
    return x

In [ ]:
OUTPUT_DIM = scan_depth * np.prod(crop_len)
model = FirstCNN(OUTPUT_DIM)

In [ ]:
print(f"The model has {count_parameters(model):,} trainable parameters.")

6138000

In [ ]:
# Loss
criterion = nn.CrossEntropyLoss() # Softmax + CrossEntropy

criterion = criterion.to(device)

In [ ]:
# Optim
optimizer = optim.Adam(model.parameters(), lr=1e-4)

model = model.to(device)

In [ ]:
N_EPOCHS = 30
train_losses, train_accs, valid_losses, valid_accs = model_training(N_EPOCHS,
                                                                    model,
                                                                    train_iterator,
                                                                    valid_iterator,
                                                                    optimizer,
                                                                    criterion,
                                                                    device,
                                                                    'lenet.pt')

In [ ]:
model_testing(model, test_loader, criterion, device, 'lenet.pt')

In [ ]:
print_report(model, test_loader, device)